## Load libraries

In [1]:
from itertools import product
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from scipy.stats import norm
import time

## Define key parameters

In [2]:
num_items = 100
num_dimensions = 5

items = ['item_' + str(x) for x in range(num_items)]
dimensions = ['dim_' + str(x) for x in range(num_dimensions)]

## Generate (random) input data

In [3]:
item_value = {item: norm(loc=10, scale=1).rvs() for item in items}

item_space_df = pd.DataFrame(list(product(*[items, dimensions])), columns=['item', 'dimension'])
item_space_df['space'] = np.round(norm(loc=10, scale=1).rvs(size=num_items*num_dimensions), 2)
item_space = item_space_df.set_index(['item', 'dimension']).to_dict()['space']

In [4]:
knapsack_space = {dimension: np.round(norm(loc=5*num_items, scale=0.5*num_items).rvs(),2) for dimension in dimensions}

## Define model via pyomo

In [5]:
def define_objective(model):
    return sum(model.a[item] * model.x[item] for item in model.Items)

def define_dimension_constraint(model, dimension):
    return sum(model.b[item, dimension] * model.x[item] for item in model.Items) <= model.c[dimension]

In [6]:
model = pyo.AbstractModel()

In [7]:
model.Items = pyo.Set(initialize=items)
model.Dimensions = pyo.Set(initialize=dimensions)

In [8]:
model.a = pyo.Param(model.Items, initialize=item_value)
model.b = pyo.Param(model.Items, model.Dimensions, initialize=item_space)
model.c = pyo.Param(model.Dimensions, initialize=knapsack_space)

In [9]:
model.x = pyo.Var(model.Items, within=pyo.Binary)

In [10]:
model.obj = pyo.Objective(rule=define_objective, sense=pyo.maximize)

In [11]:
model.dimension_constraint = pyo.Constraint(model.Dimensions, rule=define_dimension_constraint)

In [12]:
start_time = time.time()

instance = model.create_instance()

print('Instance creation: ' + str(time.time() - start_time) + ' seconds')

Instance creation: 0.020334959030151367 seconds


In [13]:
opt = pyo.SolverFactory('glpk', executable='/Applications/glpk-5.0/examples/glpsol')

## Solve the model

In [14]:
solver_start_time = time.time()

results = opt.solve(instance, tee=False, keepfiles=False)

print('Solver time: ' + str(time.time() - solver_start_time) + ' seconds')

Solver time: 0.2553677558898926 seconds


In [15]:
results.solver.termination_condition

<TerminationCondition.optimal: 'optimal'>

## Inspect the results

In [16]:
num_items = sum([1 for item in instance.Items if instance.x[item].value > 0.5])
total_value = sum([item_value[item] for item in instance.Items if instance.x[item].value > 0.5])
total_space = {dimension: sum([item_space[item, dimension]
                               for item in instance.Items if instance.x[item].value > 0.5])
               for dimension in dimensions}

print('Carry ' + str(num_items) + ' items of total value ' + str(total_value))
print(total_space)

Carry 44 items of total value 471.6977076418714
{'dim_0': 442.47999999999996, 'dim_1': 439.61999999999995, 'dim_2': 421.05, 'dim_3': 439.74, 'dim_4': 429.7800000000001}


In [17]:
print(knapsack_space)

{'dim_0': 547.99, 'dim_1': 474.99, 'dim_2': 421.08, 'dim_3': 455.37, 'dim_4': 509.86}
